In [ ]:
# %pip install -r requirements.txt

## Feature Extraction
The feature extraction pipelines are defined in this notebook. The pipeline in this notebook is used to extract the features described in the paper.

In [ ]:
import numpy as np
import neurokit2 as nk

In [ ]:
from collections import Counter

In [ ]:
from sia import Segmenter
from sia.io import read_csv, write_csv
from sia.segmenters import SlidingWindow

In [ ]:
from sia.features import extract_peaks, delineate, Waves
from sia.features.time_domain import hr, hrv, time_domain, Statistic, Feature as TimeFeature
from sia.features.frequency_domain import frequency_domain, Feature as FrequencyFeature
from sia.features.nonlinear_domain import nonlinear_domain, Feature as NonlinearFeature
from sia.features.morphology_domain import morphology_domain, Feature as MorphologyFeature

In [ ]:
SAMPLING_RATE = 1000 # Hz
WINDOW_SIZE = 60 * SAMPLING_RATE # seconds
STEP_SIZE = int(0.25 * SAMPLING_RATE) # seconds

### Feature Extraction Pipeline
The pipeline defined below segments using a Sliding Window technique and calculates the features of the data. 

In [ ]:
Segmenter() \
    .data(read_csv('./data/cleaned/*.csv', columns=['ECG_Clean', 'ECG_R_Peaks', 'category'])) \
    .segment(SlidingWindow(WINDOW_SIZE, STEP_SIZE)) \
        .skip(lambda category: len(Counter(category)) > 1) \
        .extract('category', lambda category: Counter(category).most_common(1)[0][0]) \
        .use('rpeaks', lambda ECG_R_Peaks: extract_peaks(ECG_R_Peaks)) \
        .extract(hr([Statistic.MIN, Statistic.MAX, Statistic.MEAN, Statistic.STD])) \
        .extract(hrv([Statistic.MEAN, Statistic.STD, Statistic.RMS])) \
        .extract(time_domain([TimeFeature.CVNN, TimeFeature.CVSD, TimeFeature.NN20, TimeFeature.PNN20, TimeFeature.NN50, TimeFeature.PNN50])) \
        .extract(frequency_domain([FrequencyFeature.MIN, FrequencyFeature.MAX, FrequencyFeature.MEAN, FrequencyFeature.STD, FrequencyFeature.POWER, FrequencyFeature.COVARIANCE, FrequencyFeature.ENERGY, FrequencyFeature.ENTROPY])) \
        .extract(nonlinear_domain([NonlinearFeature.ENTROPY, NonlinearFeature.POINCARE, NonlinearFeature.RQA, NonlinearFeature.FRAGMENTATION])) \
        .use('tpeaks', lambda ECG_Clean: extract_peaks(delineate(Waves.T_Peak)(ECG_Clean))) \
        .extract(morphology_domain([MorphologyFeature.TWA])) \
    .to(write_csv('./data/features/[0-9]{5}.csv'))